In [0]:
iris = spark.read.csv("/FileStore/tables/iris.csv",inferSchema=True, header=True)
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.feature import VectorAssembler
vector = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="independente")

Out[4]: VectorAssembler_2f011a92b368

In [0]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="class", outputCol="label")

In [0]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4, 5, 4, 3], featuresCol="independente", labelCol="label")

In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vector, indexer, mlp])

In [0]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
grid = ParamGridBuilder().addGrid(mlp.maxIter, [10, 100, 1000]).addGrid(mlp.layers, [[4, 5, 4, 3], [4, 4, 3]]).build()
crossval = CrossValidator(estimator=pipeline,estimatorParamMaps=grid,evaluator=MulticlassClassificationEvaluator(),numFolds=3)  

In [0]:
modelo = crossval.fit(irisTreino)

In [0]:
previsao = modelo.transform(irisTeste)
previsao.show()

+-----------+----------+-----------+----------+---------------+-----------------+-----+--------------------+--------------------+----------+
|sepallength|sepalwidth|petallength|petalwidth|          class|     independente|label|       rawPrediction|         probability|prediction|
+-----------+----------+-----------+----------+---------------+-----------------+-----+--------------------+--------------------+----------+
|        4.4|       2.9|        1.4|       0.2|    Iris-setosa|[4.4,2.9,1.4,0.2]|  1.0|[-270.68116640044...|[3.32074702659803...|       1.0|
|        4.5|       2.3|        1.3|       0.3|    Iris-setosa|[4.5,2.3,1.3,0.3]|  1.0|[-270.68116640044...|[3.32074702662823...|       1.0|
|        4.6|       3.1|        1.5|       0.2|    Iris-setosa|[4.6,3.1,1.5,0.2]|  1.0|[-270.68116640044...|[3.32074702659803...|       1.0|
|        4.6|       3.2|        1.4|       0.2|    Iris-setosa|[4.6,3.2,1.4,0.2]|  1.0|[-270.68116640044...|[3.32074702659803...|       1.0|
|        4.7|

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
performance = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)

0.9761904761904762
